In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
#This imports the necessary path to draw the single pad transmon design. 
import sys
sys.path.append('../Customized_Components')
from single_pad_transmon_pocket import TransmonPocket_Single
from LaunchpadWirebondCustom import LaunchpadWirebondCustom
from CoupledLineTee import CoupledLineTee


import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [3]:
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround

from qiskit_metal.qlibrary.tlines.mixed_path import RouteMeander, RouteMixed
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

# from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
# from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond


import pyEPR as epr

import Default_res_params as dp

from qiskit_metal.toolbox_metal.parsing import *
from collections import OrderedDict

In [4]:
import pandas as pd
import numpy as np

### Function that generates anchors for CPW. 

Currently only used to calculate the location of the short to ground component. The RouteMixed or RouteAnchors function does not work

In [5]:
def s_anchors(TQ_options, cpw_options,flip_y = False):
    if flip_y:
        factor = -1
    else:
        factor = 1
    x0 = parse_value('pos_x',TQ_options)
    y0 = parse_value('pos_y',TQ_options)
    prime_gap = parse_value('prime_gap',TQ_options)
    prime_pin = parse_value('prime_width',TQ_options)
    coupling_space = parse_value('coupling_space',TQ_options)
    down_len = parse_value('down_length',TQ_options)

    fillet = parse_value('fillet',cpw_options)
    total_len = parse_value('total_length',cpw_options)

    l = (total_len  - fillet*np.pi)/3

    angle = parse_value('orientation',TQ_options)
    angle = np.deg2rad(angle)

    # print(x0,prime_pin,prime_gap,coupling_space,down_len,l,fillet)
    # d = (prime_pin/2+prime_gap+coupling_space)#-down_len)

    x0 = x0 + (prime_pin/2+prime_gap+coupling_space+down_len)*np.sin(angle)
    y0 = y0 - (prime_pin/2+prime_gap+coupling_space+down_len)*np.cos(angle)*factor

    x1 = x0 + l*np.sin(angle)
    y1 = y0 - l*np.cos(angle)*factor

    x2 = x1 + fillet*np.cos(angle)
    y2 = y1 + fillet*np.sin(angle)*factor

    x3 = x2 - l*np.sin(angle)
    y3 = y2 + l*np.cos(angle)*factor

    x4 = x3 + fillet*np.cos(angle)
    y4 = y3 + fillet*np.sin(angle)*factor

    x5 = x4 + l*np.sin(angle)
    y5 = y4 + l*np.cos(angle)*factor

    result = OrderedDict()
    result[0] = [x1,y1]
    result[1] = [x2,y2]
    result[2] = [x3,y3]
    result[3] = [x4,y4]
    result[4] = [x5,y5]
    return result


# Initialize things by directly importing

In [6]:
gui = MetalGUI(dp.design)
design = dp.design
design.overwrite_enabled = True #This makes it possible to edit the designs later on. 

10:39PM 19s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Populating font family aliases took 173 ms. Replace uses of missing font family "Courier" with one that exists to avoid this cost. 



In [7]:
design.delete_all_components()

In [8]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60

## Specify resonator specs

Hand measured using kLayout

In [9]:
coupling_d = [100,105,140,200,2,15,40,75]
curve_d = [380,380,380,400,300,310,330,340]
ground_plane_d = 10
fillets = [200,200,280,320,200,200,200,240]

prime_pin = 14.18
prime_gap = 8

res_orientation = [(-1)**i*90 for i in range(8)]
res_flip_y = [i%2==0 for i in range(8)]

res_order = [4,2,6,0, 7,1,5,3]


# Put down wirebond pads

In [10]:
wb_x0 = 4.7/2
wb_y0 = 3.5-1.86

In [11]:
# # -*- coding: utf-8 -*-

# # This code is part of Qiskit.
# #
# # (C) Copyright IBM 2017, 2021.
# #
# # This code is licensed under the Apache License, Version 2.0. You may
# # obtain a copy of this license in the LICENSE.txt file in the root directory
# # of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
# #
# # Any modifications or derivative works of this code must retain this
# # copyright notice, and modified files need to carry a notice indicating
# # that they have been altered from the originals.

# #  This a launch structure used on BlueJayV2, used for wire bonding
# #  There is no CPW tee attached to this p#

# # Imports required for drawing

# # import numpy as np # (currently not used, may be needed later for component customization)
# from qiskit_metal import draw
# from qiskit_metal.toolbox_python.attr_dict import Dict
# from qiskit_metal.qlibrary.core import QComponent

# # Define class and options for the launch geometry


# class LaunchpadWirebondCustom(QComponent):
#     """Launch pad to feed/read signals to/from the chip.

#     Inherits 'QComponent' class.

#     .. image::
#         LaunchpadWirebond.png

#     .. meta::
#         Launchpad Wirebond

#     Creates a 50 ohm launch pad with a ground pocket cutout.
#     Limited but expandable parameters to control the launchpad polygons.
#     The (0,0) point is the center of the necking of the launch tip.
#     The pin attaches directly to the built in lead length at its midpoint

#     Pocket and pad:
#         Pocket and launch pad geometries are currently fixed.
#         (0,0) point is the midpoint of the necking of the launch tip.
#         Pocket is a negative shape that is cut out of the ground plane

#     Values (unless noted) are strings with units included, (e.g., '30um')

#     Sketch:
#         Below is a sketch of the launch
#         ::

#             -----------
#             |          \
#             |      ---------\\
#             |      |    0    |    (0,0) pin at midpoint of necking, before the lead
#             |      ---------//
#             |          /
#             -----------

#             y
#             ^
#             |
#             |------> x

#     .. image::
#         LaunchpadWirebond.png

#     Default Options:
#         * trace_width: 'cpw_width' -- Width of the transmission line attached to the launch pad
#         * trace_gap: 'cpw_gap' -- Gap of the transmission line
#         * lead_length: '25um' -- Length of the transmission line attached to the launch pad
#         * pad_width: '80um' -- Width of the launch pad
#         * pad_height: '80um' -- Height of the launch pad
#         * pad_gap: '58um' -- Gap of the launch pad
#         * taper_height: '122um' -- Height of the taper from the launch pad to the transmission line
#         * pin_space: '100um' -- Space between the pin and the launch pad on the direction away from the transmission line
#     """

#     default_options = Dict(trace_width='trace_width',
#                            trace_gap='trace_gap',
#                            lead_length='25um',
#                            pad_width='80um',
#                            pad_height='80um',
#                            pad_gap='58um',
#                            taper_height='122um',
#                            pin_space = '100um')
#     """Default options"""

#     TOOLTIP = """Launch pad to feed/read signals to/from the chip."""

#     def make(self):
#         """This is executed by the user to generate the qgeometry for the
#         component."""

#         p = self.p

#         pad_width = p.pad_width
#         pad_height = p.pad_height
#         pad_gap = p.pad_gap
#         trace_width = p.trace_width
#         trace_width_half = trace_width / 2.
#         pad_width_half = pad_width / 2.
#         lead_length = p.lead_length
#         taper_height = p.taper_height
#         trace_gap = p.trace_gap

#         pad_gap = p.pad_gap
#         pin_space = p.pin_space
#         #########################################################

#         # Geometry of main launch structure
#         # The shape is a polygon and we prepare this point as orientation is 0 degree
#         launch_pad = draw.Polygon([
#             (0, trace_width_half), 
#             (-taper_height, pad_width_half),
#             (-(pad_height + taper_height), pad_width_half),
#             (-(pad_height + taper_height), -pad_width_half),
#             (-taper_height, -pad_width_half), (0, -trace_width_half),
#             (lead_length, -trace_width_half), (lead_length, trace_width_half),
#             (0, trace_width_half)
#         ])

#         # Geometry pocket (gap)
#         # Same way applied for pocket
#         pocket = draw.Polygon([(0, trace_width_half + trace_gap),
#                                (-taper_height, pad_width_half + pad_gap),
#                                (-(pad_height + taper_height + pin_space),
#                                 pad_width_half + pad_gap),
#                                (-(pad_height + taper_height + pin_space),
#                                 -(pad_width_half + pad_gap)),
#                                (-taper_height, -(pad_width_half + pad_gap)),
#                                (0, -(trace_width_half + trace_gap)),
#                                (lead_length, -(trace_width_half + trace_gap)),
#                                (lead_length, trace_width_half + trace_gap),
#                                (0, trace_width_half + trace_gap)])

#         # These variables are used to graphically locate the pin locations
#         main_pin_line = draw.LineString([(lead_length, trace_width_half),
#                                          (lead_length, -trace_width_half)])

#         # Create polygon object list
#         polys1 = [main_pin_line, launch_pad, pocket]

#         # Rotates and translates all the objects as requested. Uses package functions in
#         # 'draw_utility' for easy rotation/translation
#         polys1 = draw.rotate(polys1, p.orientation, origin=(0, 0))
#         polys1 = draw.translate(polys1, xoff=p.pos_x, yoff=p.pos_y)
#         [main_pin_line, launch_pad, pocket] = polys1

#         # Adds the object to the qgeometry table
#         self.add_qgeometry('poly', dict(launch_pad=launch_pad), layer=p.layer)

#         # Subtracts out ground plane on the layer its on
#         self.add_qgeometry('poly',
#                            dict(pocket=pocket),
#                            subtract=True,
#                            layer=p.layer)

#         # Generates the pins
#         self.add_pin('tie', main_pin_line.coords, trace_width)


In [12]:
Wirebond_opt = Dict(trace_width=design.variables['trace_width'],
                           trace_gap=design.variables['trace_gap'],
                           lead_length='30um',
                           pad_width='200um',
                           pad_height='500um',
                           pad_gap='100um',
                           taper_height='100um',
                           layer = str(qubit_layer),
                           pin_space = '200um')

In [13]:
left_lauch_pads = LaunchpadWirebondCustom(design,'wb_left', options = Dict(pos_x = '{}mm'.format(-wb_x0),pos_y = '{}mm'.format(-wb_y0),orientation = '90',**Wirebond_opt))
right_lauch_pads = LaunchpadWirebondCustom(design,'wb_right', options = Dict(pos_x = '{}mm'.format(wb_x0),pos_y = '{}mm'.format(-wb_y0),orientation = '90',**Wirebond_opt))

gui.rebuild()
gui.autoscale()


# Put Down Resonator with Coupled Line Tee

Import resonator parameters. Calculated using coupler_optimization.ipynb with website to optimize frequency and impedance. All the other parameters are measured from dxf file

In [14]:
resonator_params = pd.read_csv('results.csv')
print(resonator_params)

   Unnamed: 0  Frequency          Q  Length  Gap    Pin  coupling_gap
0           0        5.9  1278612.0  5.0535   10  17.73           118
1           1        6.2  1210136.0  4.8072   12  21.27           123
2           2        6.5  1149617.0  4.5836   14  24.82           158
3           3        6.7  1113073.0  4.4457   16  28.37           158
4           4        7.0  1063157.0  4.2536    2   3.55            20
5           5        7.2  1032764.0  4.1344    4   7.09            33
6           6        7.5   990927.0  3.9674    6  10.64            58
7           7        7.9   941290.0  3.7646    8  14.18            93


In [15]:
freqs = resonator_params['Frequency'].values
Qs = resonator_params['Q'].values
gaps = resonator_params['Gap'].values
pins = resonator_params['Pin'].values
coupling_gap = resonator_params['coupling_gap'].values
lengths = resonator_params['Length'].values

Define locations of resonators by defining location of the coupled line tee

In [16]:
res_x = 1.15
delta_x = 3.5-res_x

res_x_all  = np.array([-delta_x*(-1)**i for i in range(8)])

res_y_all = np.array([(5254+5210)/2,(4301+4248)/2,(3448+3387)/2,
             (2395+2325)/2,(2365+2356)/2,(3326+3309)/2,
             (4338+4311)/2,(5249+5214)/2])/1000-3.5

In [17]:
# # -*- coding: utf-8 -*-

# # This code is part of Qiskit.
# #
# # (C) Copyright IBM 2017, 2021.
# #
# # This code is licensed under the Apache License, Version 2.0. You may
# # obtain a copy of this license in the LICENSE.txt file in the root directory
# # of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
# #
# # Any modifications or derivative works of this code must retain this
# # copyright notice, and modified files need to carry a notice indicating
# # that they have been altered from the originals.

# from qiskit_metal import draw, Dict
# from qiskit_metal.qlibrary.core import QComponent
# import numpy as np


# class CoupledLineTee(QComponent):
#     """Generates a three pin (+) structure comprised of a primary two pin CPW
#     transmission line, and a secondary one pin neighboring CPW transmission
#     line that is capacitively/inductively coupled to the primary. Such a
#     structure can be used, as an example, for generating CPW resonator hangars
#     off of a transmission line.

#     Inherits QComponent class.

#     ::

#         +----------------------------+
#         ------------------------------
#         |
#         |
#         |
#         |
#         +

#     .. image::
#         CoupledLineTee.png

#     .. meta::
#         Coupled Line Tee

#     Default Options:
#         * prime_width: '10um' -- The width of the trace of the two pin CPW transmission line
#         * prime_gap: '6um' -- The dielectric gap of the two pin CPW transmission line
#         * second_width: '10um' -- The width of the trace of the one pin CPW transmission line
#         * second_gap: '6um' -- The dielectric gap of the one pin CPW transmission line
#         * coupling_space: '3um' -- The amount of ground plane between the two transmission lines
#         * coupling_length: '100um' -- The length of parallel between the two transmission lines
#           note: this includes the distance of the curved second of the second line
#         * down_length: '100um' -- The length of the hanging part of the resonator, including the etched region
#         * fillet: '25um'
#         * mirror: False -- Flips the hanger around the y-axis
#         * open_termination: True -- sets if the termination of the second line at the coupling side
#           is an open to ground or short to ground
#     """
#     component_metadata = Dict(short_name='cpw', _qgeometry_table_path='True')
#     """Component metadata"""

#     #Currently setting the primary CPW length based on the coupling_length
#     #May want it to be it's own value that the user can control?
#     default_options = Dict(prime_width='10um',
#                            prime_gap='6um',
#                            prime_length = '120um',
#                            second_width='10um',
#                            second_gap='6um',
#                            coupling_space='3um',
#                            pin_drop = '0um',
#                            coupling_length='100um',
#                            down_length='100um',
#                            fillet='25um',
#                            mirror=False,
#                            open_termination=True)
#     """Default connector options"""

#     TOOLTIP = """Generates a three pin (+) 
#     structure comprised of a primary two 
#     pin CPW transmission line, and a 
#     secondary one pin neighboring CPW 
#     transmission line that is 
#     capacitively/inductively coupled 
#     to the primary."""

#     def make(self):
#         """Build the component."""
#         p = self.p

#         prime_cpw_length = p.prime_length
#         second_flip = 1
#         if p.mirror:
#             second_flip = -1

#         #Primary CPW
#         prime_cpw = draw.LineString([[-prime_cpw_length / 2, 0],
#                                      [prime_cpw_length / 2, 0]])
        
#         second_termination = 0
#         if p.open_termination:
#             second_termination = p.second_gap

#         #Secondary CPW
#         second_down_length = p.down_length
#         second_y = -p.prime_width / 2 - p.prime_gap  - p.coupling_space
#         second_cpw = draw.LineString(
#             [[0, second_y-p.pin_drop],
#              [0,second_y - second_down_length]])

        

#         second_cpw_etch = draw.LineString(
#             [[0, second_y],
#              [0,second_y - second_down_length]])

#         #Rotate and Translate
#         c_items = [prime_cpw, second_cpw, second_cpw_etch]
#         c_items = draw.rotate(c_items, p.orientation, origin=(0, 0))
#         c_items = draw.translate(c_items, p.pos_x, p.pos_y)
#         [prime_cpw, second_cpw, second_cpw_etch] = c_items

#         #Add to qgeometry tables
#         self.add_qgeometry('path', {'prime_cpw': prime_cpw},
#                            width=p.prime_width)
#         self.add_qgeometry('path', {'prime_cpw_sub': prime_cpw},
#                            width=p.prime_width + 2 * p.prime_gap,
#                            subtract=True)
#         self.add_qgeometry('path', {'second_cpw': second_cpw},
#                            width=p.second_width,
#                            fillet=p.fillet)
#         self.add_qgeometry('path', {'second_cpw_sub': second_cpw_etch},
#                            width=p.second_width + 2 * p.second_gap,
#                            subtract=True,
#                            fillet=p.fillet)

#         #Add pins
#         prime_pin_list = prime_cpw.coords
#         second_pin_list = second_cpw.coords

#         self.add_pin('prime_start',
#                      points=np.array(prime_pin_list[::-1]),
#                      width=p.prime_width,
#                      input_as_norm=True)
#         self.add_pin('prime_end',
#                      points=np.array(prime_pin_list),
#                      width=p.prime_width,
#                      input_as_norm=True)
#         self.add_pin('second_end',
#                      points=np.array(second_pin_list),
#                      width=p.second_width,
#                      input_as_norm=True)


pre define some options

In [18]:
TQ_options = dp.TQ_options
TQ_options['prime_length'] = '120um'
TQ_options['prime_gap'] = design.variables['trace_gap']
TQ_options['prime_width'] = design.variables['trace_width']

## Actual code that puts down CPW and CoupledLineTee

NOTE: need to fix code and use RouteMixed with anchors to route CPWs. Currently using RouteMeander and lack control. 

In [19]:
TQs = []
short_to_grounds = []
cpws = []

connection_cpws = []

for i in range(8):
    curve_d_i = curve_d[i]
    coupling_d_i = coupling_d[i]
    pins_i = pins[i]
    gap_i = gaps[i]
    fillet = fillets[i]

    down_len =  - prime_pin/2 - prime_gap + curve_d_i+fillet*np.pi/2

    TQ_options['down_length'] =  str(down_len) + 'um'
    TQ_options['coupling_space'] = str(ground_plane_d) + 'um'
    TQ_options['second_width'] = str(pins[i]) + 'um'
    TQ_options['second_gap'] = str(gaps[i]) + 'um'
    # TQ_options['coupling_space'] = str(coupling_gap[i])+ 'um'
    TQ_options['pin_drop'] = str(coupling_d[i])+'um'
    TQ_options['orientation'] = str(res_orientation[i])


    TQs.append(CoupledLineTee(design, 'TQ_'+str(i), options=dict(pos_x='{} mm'.format(res_x_all[i]),
                                                pos_y='{} mm'.format(res_y_all[i]), **TQ_options)))

    CPW_options = dp.CPW_options
    CPW_options['pin_width'] = str(pins[i]) + 'um'
    CPW_options['gap_width'] = str(gaps[i]) + 'um'
    CPW_options['total_length'] = str(np.around(lengths[i]-down_len/1000,4)) + 'mm'
    CPW_options['pin_inputs'] = in_inputs=Dict(
                start_pin=Dict(component='TQ_'+str(i), pin='second_end'),
                end_pin=Dict(component='short'+str(i), pin='short'),)

    CPW_options['trace_gap'] = str(gaps[i]) + 'um'
    CPW_options['trace_width'] = str(pins[i]) + 'um'
    CPW_options['lead'] = dict(start_straight = '600 um', end_straight = '400 um')
    CPW_options['fillet'] = '{} um'.format(fillet)
    CPW_options['prevent_short_edges'] = False
    CPW_options['snap'] = True

    anchors = s_anchors(TQs[-1].options, CPW_options,flip_y=res_flip_y[i])
    orientation = parse_value('orientation', TQ_options) + 270
    short_to_grounds.append(ShortToGround(design, 'short'+str(i), options=dict(pos_x='{} mm'.format(anchors[4][0]), pos_y='{} mm'.format(anchors[4][1]), 
                                                        orientation='{}'.format(orientation))))
 
    CPW_options['fillet'] = '{} um'.format(fillet/2.1)
    CPW_options['anchor'] = anchors
    CPW_options['meander'] = dict(spacing= '{}um'.format(fillet), asymmetry= '0um')

    cpws.append(RouteMeander(design, 'cpw'+str(i), options = CPW_options))
                
    

    gui.rebuild()
    gui.autoscale()

### Connect coupled line tees

In [20]:
for i, cpw in enumerate(res_order):
        if i ==7:
                break
        elif i == 3:
                continue
        else:
                trans_ops = dp.trans_options
                trans_ops['pin_inputs'] = {'start_pin': {'component': TQs[cpw].name, 
                                                        'pin': 'prime_end'},
                                        'end_pin': {'component': TQs[res_order[i+1]].name, 
                                                        'pin': 'prime_start'}}
                connection_cpws.append(RouteStraight(design, 
                                                'connection_cpw'+str(cpw), 
                                                options = Dict(**dp.trans_options)))
                gui.rebuild()

In [21]:
anchors = OrderedDict()
anchors[0] = (3.5-1.230,-0.2)
anchors[1] = (3.5-1.230,0.2)

In [22]:
i = 3
trans_ops1 = dp.trans_options
trans_ops1['lead'] = dict(start_straight = '300 um', end_straight = '300 um')
trans_ops1['pin_inputs'] = {'start_pin': {'component': TQs[res_order[i]].name, 'pin': 'prime_end'},
                           'end_pin': {'component': TQs[res_order[i+1]].name, 'pin': 'prime_start'}}
trans_ops1['fillet'] = '90um'
total_len = 0.3*2+4.7 + 0.9*(np.pi-2)
trans_ops1['total_length'] = '{}mm'.format(total_len)
cpw_top = RouteMixed(design, 'connection_cpw'+str(i),options = Dict(anchor = anchors,**trans_ops1))

gui.rebuild()
                

In [23]:

trans_ops['pin_inputs'] = {'end_pin': {'component':'TQ_4','pin': 'prime_start'},'start_pin': {'component': 'wb_left', 'pin': 'tie'}}
# connection_cpws.append(RouteStraight(design,'connection_cpw'+str(cpw), options = Dict(**dp.trans_options)))

cpw_left_launch = RouteStraight(design, 'cpw_left_launch', options = Dict(**trans_ops))

trans_ops['pin_inputs'] = {'end_pin': {'component':'TQ_3','pin': 'prime_end'},'start_pin': {'component': 'wb_right', 'pin': 'tie'}}
cpw_right_launch = RouteStraight(design, 'cpw_right_launch', options = Dict(**trans_ops))

gui.rebuild()

In [24]:
# design.chips.main.size['size_x'] = '10mm'
# design.chips.main.size['size_y'] = '10mm'
# gui.rebuild()
a_gds = design.renderers.gds

In [25]:
a_gds.options.cheese.edge_nocheese = '200um'
a_gds.options.no_cheese.buffer = '16um'
a_gds.options.cheese.cheese_1_radius = '70um'

In [26]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60

In [27]:
a_gds.options['max_points'] = 500
a_gds.options['fabricate'] = True

In [28]:
a_gds.options['cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['no_cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['cheese']['view_in_file']['main'][junction_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][junction_layer] = True
a_gds.options['cheese']['view_in_file']['main'][ab_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_layer] = True
a_gds.options['cheese']['view_in_file']['main'][ab_square_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_square_layer] = True
a_gds.options['cheese']['view_in_file']['main'][1] = True
a_gds.options['no_cheese']['view_in_file']['main'][1] = True

In [29]:
# a_gds.options['cheese']['view_in_file']['main'][qubit_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][qubit_layer] = True
# a_gds.options['cheese']['view_in_file']['main'][junction_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][junction_layer] = True
# a_gds.options['cheese']['view_in_file']['main'][ab_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][ab_layer] = True
# a_gds.options['cheese']['view_in_file']['main'][ab_square_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][ab_square_layer] = True
# a_gds.options['cheese']['view_in_file']['main']['1'] = True
# a_gds.options['no_cheese']['view_in_file']['main']['1'] = True



a_gds.options['fabricate'] = False
a_gds.options.tolerance = '0.0000005'
a_gds.options.precision = '0.0000000005'

In [30]:
a_gds.options['cheese']

{'datatype': '100',
 'shape': '0',
 'cheese_0_x': '25um',
 'cheese_0_y': '25um',
 'cheese_1_radius': '70um',
 'view_in_file': {'main': {1: True, 5: True, 20: False, 31: False, 30: False}},
 'delta_x': '100um',
 'delta_y': '100um',
 'edge_nocheese': '200um'}

In [31]:
a_gds.options.cheese.edge_nocheese = '150um'
a_gds.options.no_cheese.buffer = '200um'
a_gds.options.cheese.cheese_1_radius = '100um'

In [32]:
# gui.rebuild()
a_gds.export_to_gds('resonator_design.gds')

10:39PM 26s WARNING [_import_junction_gds_file]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"/Users/wendy/Desktop/Wendy-qiskit-code/resources".


ground_main_5


1